In [ ]:
pip install pymysql

In [ ]:
import json
import csv
import numpy as np
import pandas as pd
import sqlalchemy
from urllib.parse import quote

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [ ]:
connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.format('USER',quote('PASS'),'IP','DB'))

In [ ]:
data = pd.read_sql("""
    SELECT `selectedFeatures` FROM `test_case_10` ORDER BY `test_case_10`.`test_accuracy` DESC limit 15;
""", con=connection)


featureList=list();
for index, row in data.iterrows():
    featureList=featureList+np.hstack(json.loads(row["selectedFeatures"])).tolist()


unique, counts = np.unique(np.hstack(featureList), return_counts=True)
sortValue=np.asarray((unique, counts)).T
SelectedFeatures=sortValue[np.flip(np.argsort(sortValue[:, 1]))]

dataframe=pd.DataFrame(SelectedFeatures, columns=['name','total']) 
dataframe.head(30)

In [ ]:
data = pd.read_sql("""
    SELECT `selectedFeatures` FROM `test_case_15` ORDER BY `test_case_15`.`test_accuracy` DESC limit 15;
""", con=connection)


featureList=list();
for index, row in data.iterrows():
    featureList=featureList+np.hstack(json.loads(row["selectedFeatures"])).tolist()


unique, counts = np.unique(np.hstack(featureList), return_counts=True)
sortValue=np.asarray((unique, counts)).T
SelectedFeatures=sortValue[np.flip(np.argsort(sortValue[:, 1]))]

print(SelectedFeatures[:,1].astype(int).sum())
SelectedFeatures

dataframe=pd.DataFrame(SelectedFeatures, columns=['name','total']) 
dataframe.head(30)

In [ ]:
data = pd.read_sql("""
    SELECT `selectedFeatures` FROM `test_case_20` ORDER BY `test_case_20`.`test_accuracy` DESC limit 15;
""", con=connection)

featureList=list();
for index, row in data.iterrows():
    featureList=featureList+np.hstack(json.loads(row["selectedFeatures"])).tolist()


unique, counts = np.unique(np.hstack(featureList), return_counts=True)
sortValue=np.asarray((unique, counts)).T
SelectedFeatures=sortValue[np.flip(np.argsort(sortValue[:, 1]))]

print(SelectedFeatures[:,1].astype(int).sum())
SelectedFeatures

dataframe=pd.DataFrame(SelectedFeatures, columns=['name','total']) 
dataframe.head(30)

In [ ]:
data = pd.read_sql('''
SELECT 
    `selectionMethod`, 
    count(*),
    ROUND(MAX(`test_accuracy`) * 100, 2) as "MAX_ACCURACY", 
    ROUND(MIN(`test_accuracy`) * 100, 2) as "MIN_ACCURACY", 
    ROUND(AVG(`test_accuracy`) * 100, 2) as "AVG_ACCURACY", 
    ROUND(AVG(`fit_time`), 2) as "AVG_FIT", 
    ROUND(AVG(`score_time`), 2) as "AVG_SCORE" 
FROM 
    `test_case_10` 
where
    `modelName` not in ('svc_n','knn5','knn3')
GROUP BY 
    `selectionMethod` 
ORDER BY 
    `test_case_10`.`selectionMethod` ASC;

''', con=connection)

data.head(10)

,selectionMethod,count(*),MAX_ACCURACY,MIN_ACCURACY,AVG_ACCURACY,AVG_FIT,AVG_SCORE
0,f_classif,70,86.46,75.43,81.94,123.15,7.52
1,mutual_info_classif,70,98.48,67.01,91.84,315.85,5.10
2,rfe_DecisionTree,70,98.64,69.76,92.23,535.74,4.87
3,sf_RandomForest,70,98.47,67.39,91.87,276.53,5.02
4,sf_XGBClassifier,70,98.21,66.84,91.24,123.26,4.41


In [ ]:
data = pd.read_sql('''
SELECT
	`modelName`,
	ROUND((`test_accuracy`) * 100, 2), 
    ROUND((`test_roc_auc`) * 100, 2),
    ROUND((`test_f1`) * 100, 2), 
    ROUND((`test_recall`) * 100, 2), 
    ROUND((`test_Precision`) * 100, 2)
FROM
    `sampling_cross_validation`
WHERE
    `samplingModel`="SMOTETomek"  
ORDER BY `sampling_cross_validation`.`modelName` ASC;
''', con=connection)

data.head(20)

,modelName,"ROUND((`test_accuracy`) * 100, 2)","ROUND((`test_roc_auc`) * 100, 2)","ROUND((`test_f1`) * 100, 2)","ROUND((`test_recall`) * 100, 2)","ROUND((`test_Precision`) * 100, 2)"
0,ab,99.70,99.99,99.70,99.65,99.75
1,bc_100,99.81,99.98,99.81,99.83,99.80
2,dt,99.71,99.71,99.71,99.72,99.71
3,et_100,99.87,100.00,99.87,99.86,99.89
4,et_500,99.88,100.00,99.88,99.89,99.88
5,gn,90.03,98.48,89.29,82.87,96.79
6,knn3,99.78,99.91,99.78,99.71,99.84
7,knn5,99.74,99.93,99.74,99.60,99.87
8,lr,92.94,95.81,92.83,91.18,94.55
9,rf_100,99.86,100.00,99.86,99.91,99.82


In [ ]:
data = pd.read_sql('''
    SELECT 
      `modelName`, 
      `outLierModel`, 
      `removedSample`, 
      ROUND((`test_accuracy`) * 100, 2), 
      ROUND((`test_roc_auc`) * 100, 2), 
      ROUND((`test_f1`) * 100, 2), 
      ROUND((`test_recall`) * 100, 2), 
      ROUND((`test_Precision`) * 100, 2) 
    FROM 
      `outlier_cross_validation` 
    where 
      `modelName` not in ('ab','bc_100','dt_100','dt','et_100','et_500','knn3','knn5','rf_100','rf_500','svc_n') and 
      `outLierModel`<>'iso_1' 
    ORDER BY 
      `outlier_cross_validation`.`modelName` ASC, 
      `outlier_cross_validation`.`outLierModel` ASC;
''', con=connection)


data.head(25)

,modelName,outLierModel,removedSample,"ROUND((`test_accuracy`) * 100, 2)","ROUND((`test_roc_auc`) * 100, 2)","ROUND((`test_f1`) * 100, 2)","ROUND((`test_recall`) * 100, 2)","ROUND((`test_Precision`) * 100, 2)"
0,gn,all,0,89.54,97.67,86.27,79.01,95.00
1,gn,ee_01,994,90.04,97.92,87.04,79.86,95.64
2,gn,ee_05,4970,90.80,98.27,87.70,80.21,96.73
3,gn,iso_01,994,89.58,97.68,86.19,78.88,94.99
4,gn,iso_05,4970,89.87,97.88,86.11,78.34,95.60
5,gn,lof,2679,89.81,97.72,86.22,78.96,94.95
6,gn,ncSVM_01,2982,90.93,97.86,88.04,82.36,94.55
7,gn,ncSVM_05,4974,91.34,98.14,88.66,82.69,95.56
8,lr,all,0,91.28,94.43,89.12,85.82,92.68
9,lr,ee_01,994,91.19,94.51,89.07,85.87,92.52


In [ ]:
data = pd.read_sql('''
SELECT `DataSet`, ROUND((`Test_AUC`) * 100, 2) FROM `sampling_gene` WHERE `samplingModel` LIKE 'SMOTEENN' AND `modelName` LIKE 'et_500';
''', con=connection)
data.head(25)

,DataSet,"ROUND((`Test_AUC`) * 100, 2)"
0,DS4,65.90
1,DS5,53.99
2,DS6,70.09
3,DS7,91.60
4,DS12,70.46
5,DS13,62.70
6,DS14,81.90
7,DS15,94.13
8,DS16,84.49


In [ ]:
data = pd.read_sql('''
    SELECT 
        `MainDataSet`, 
        ROUND(AVG(`Test_AUC`) * 100, 2) 
    FROM 
        `comparison_test_models` 
    WHERE 
        `modelName` LIKE 'et_500' AND `samplingModel` LIKE 'SMOTEENN' AND `outlierModel` LIKE 'iso' and `DataSet`<> `MainDataSet` 
    group by `MainDataSet`;
''', con=connection)
data.head(25)

,MainDataSet,"ROUND(AVG(`Test_AUC`) * 100, 2)"
0,DS4,69.18
1,DS5,53.72
2,DS6,55.26
3,DS7,60.02
4,DS12,60.15
5,DS13,60.84
6,DS14,64.14
7,DS15,67.18
8,DS16,59.63


In [ ]:
data = pd.read_sql('''
    SELECT 
        `MainDataSet`, 
        ROUND(AVG(`Test_AUC`) * 100, 2) 
    FROM 
        `comparison_test_models` 
    WHERE 
        `modelName` LIKE 'et_500' AND `samplingModel` LIKE 'SMOTEENN' AND `outlierModel` LIKE 'iso' and `DataSet`<> `MainDataSet` 
    group by `MainDataSet`;
''', con=connection)
data.head(25)

,MainDataSet,"ROUND(AVG(`Test_AUC`) * 100, 2)"
0,DS4,69.83
1,DS5,52.12
2,DS6,53.25
3,DS7,60.37
4,DS12,59.16
5,DS13,63.25
6,DS14,65.51
7,DS15,65.72
8,DS16,58.80


In [ ]:
data = pd.read_sql('''
SELECT
	`MainDataSet`,
    `DataSet`,
    ROUND((`Test_AUC`) * 100, 2)
FROM
    `comparison_test_models`
WHERE
    `modelName` LIKE 'et_500' AND `samplingModel` LIKE 'SMOTEENN' AND `outlierModel` LIKE 'all'
''', con=connection)
data.head(100)

,MainDataSet,DataSet,"ROUND((`Test_AUC`) * 100, 2)"
0,DS4,DS4,100.00
1,DS4,DS5,66.14
2,DS4,DS6,51.50
3,DS4,DS7,82.31
4,DS4,DS12,87.16
5,DS4,DS13,52.26
6,DS4,DS14,69.70
7,DS4,DS15,71.07
8,DS4,DS16,78.53
9,DS5,DS4,62.27


In [ ]:
data = pd.read_sql('''
    SELECT `modelName`, ROUND(AVG(`Test_AUC`) * 100,2) as 'Test_AUC' FROM `sampling_gene` WHERE `samplingModel`='SMOTEENN' GROUP BY `modelName`, `samplingModel` ORDER BY Test_AUC DESC;
''', con=connection)
data.head(100)

,modelName,Test_AUC
0,svc_n,75.75
1,et_500,75.03
2,knn5,74.73
3,et_100,74.66
4,knn3,74.40
5,rf_500,73.37
6,dt,73.20
7,rf_100,72.91
8,gn,72.73
9,ab,72.33
